<a href="https://colab.research.google.com/github/chi-yan/notebooks/blob/master/Classifier_builder_with_PyCaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Multi-label classification with pycaret

Code predicts "JOB CATEGORY" from a given description

Pycaret used to evaluate different models

Data source: https://data.world/promptcloud/30000-job-postings-from-seek-australia (hope's that ok...)

To use:
- download seek_australia.csv from https://data.world/promptcloud/30000-job-postings-from-seek-australia and save to Google Drive 



In [4]:
#https://data.world/promptcloud/30000-job-postings-from-seek-australia

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/seek_australia.csv')


In [3]:
#drop rows where either category or description is NAN

In [6]:
data.dropna(subset = ["category","job_description"], inplace=True)
data['category'] = data['category'].astype('category')


In [7]:
categories = list(data['category'].cat.codes)

In [8]:
dict( enumerate(data['category'].cat.categories ) )
d = dict( enumerate(data['category'].cat.categories ) )
print(d)

{0: 'Accounting', 1: 'Administration & Office Support', 2: 'Advertising, Arts & Media', 3: 'Banking & Financial Services', 4: 'CEO & General Management', 5: 'Call Centre & Customer Service', 6: 'Community Services & Development', 7: 'Construction', 8: 'Consulting & Strategy', 9: 'Design & Architecture', 10: 'Education & Training', 11: 'Engineering', 12: 'Farming, Animals & Conservation', 13: 'Government & Defence', 14: 'Healthcare & Medical', 15: 'Hospitality & Tourism', 16: 'Human Resources & Recruitment', 17: 'Information & Communication Technology', 18: 'Insurance & Superannuation', 19: 'Legal', 20: 'Manufacturing, Transport & Logistics', 21: 'Marketing & Communications', 22: 'Mining, Resources & Energy', 23: 'Real Estate & Property', 24: 'Retail & Consumer Products', 25: 'Sales', 26: 'Science & Technology', 27: 'Self Employment', 28: 'Sport & Recreation', 29: 'Trades & Services'}


In [9]:
descriptions = data['job_description']

In [10]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


In [11]:
len(categories)

29655

In [12]:
embeddings = embed(descriptions[0:10000])


In [13]:
x = embeddings
y = categories[0:10000]

In [14]:
import numpy as np

x_np = np.array(x)
y_np = np.array(y)

In [15]:
x_np

array([[-0.02446891, -0.06791084,  0.04574436, ...,  0.01000977,
        -0.06279211, -0.04679998],
       [ 0.02802009, -0.05871655,  0.05011086, ..., -0.03657727,
        -0.05850006, -0.02305215],
       [-0.05585853, -0.06168897, -0.05547146, ...,  0.01977861,
        -0.06100542, -0.04146077],
       ...,
       [-0.06100704, -0.06207096,  0.05299966, ..., -0.06170482,
        -0.05043088, -0.02256734],
       [ 0.00889324, -0.06049362,  0.01974255, ...,  0.03079445,
        -0.05770535, -0.01278769],
       [-0.05476727, -0.06326474, -0.01913269, ..., -0.02954619,
        -0.04700347, -0.04186895]], dtype=float32)

In [14]:
#embeddings = embed(list(descriptions[0:50]))

In [15]:
!pip install scikit-multilearn

In [16]:
# Import BinaryRelevance from skmultilearn
from skmultilearn.problem_transform import BinaryRelevance

# Import SVC classifier from sklearn
from sklearn.svm import SVC

# Setup the classifier
classifier = BinaryRelevance(classifier=SVC(), require_dense=[False,True]) #support vector classifier

# Train
classifier.fit(x_np,y_np)

# Predict
#y_pred = classifier.predict(X_test)

BinaryRelevance(classifier=SVC(), require_dense=[False, True])

In [17]:
y_pred = classifier.predict(x_np)

In [18]:
y_np

array([24, 13, 29, ...,  6, 13, 29])

In [19]:
y_numpy = np.array(y_np)

In [20]:
yl = list(y_numpy)  #OBSERVED LABELS in list form

In [21]:
y_pred_dense = list(y_pred.todense().ravel().flatten())

In [22]:
yp = y_pred_dense 
print(yp[0].tolist())

[[24, 17, 29, 29, 29, 13, 25, 6, 14, 17, 29, 20, 20, 11, 7, 29, 22, 1, 21, 11, 1, 20, 3, 21, 17, 14, 29, 20, 21, 20, 14, 20, 13, 29, 24, 10, 29, 29, 29, 7, 17, 13, 0, 16, 10, 0, 14, 29, 29, 25, 11, 23, 10, 7, 29, 6, 20, 10, 6, 29, 1, 28, 13, 5, 25, 19, 17, 5, 23, 29, 17, 14, 29, 11, 29, 14, 13, 14, 14, 5, 5, 20, 0, 29, 6, 10, 1, 20, 13, 29, 29, 0, 29, 16, 21, 20, 17, 13, 14, 29, 13, 1, 17, 21, 29, 25, 15, 9, 15, 14, 16, 17, 14, 1, 10, 22, 14, 17, 14, 13, 24, 17, 1, 0, 15, 29, 11, 7, 21, 29, 17, 29, 1, 17, 29, 12, 3, 5, 0, 10, 20, 29, 29, 13, 3, 15, 6, 24, 1, 17, 25, 17, 20, 1, 0, 1, 23, 20, 17, 14, 9, 5, 1, 16, 11, 14, 13, 14, 21, 22, 17, 17, 13, 6, 29, 1, 29, 14, 15, 10, 21, 14, 7, 0, 3, 29, 13, 23, 14, 13, 3, 0, 15, 11, 13, 20, 22, 24, 29, 0, 13, 29, 18, 24, 21, 0, 14, 17, 15, 17, 25, 10, 25, 20, 1, 20, 14, 1, 29, 7, 29, 1, 14, 13, 14, 17, 1, 24, 14, 7, 13, 20, 1, 29, 7, 13, 17, 14, 10, 7, 29, 13, 13, 1, 29, 21, 25, 20, 23, 11, 25, 13, 25, 15, 16, 12, 17, 17, 13, 1, 17, 19, 15, 17, 2

In [23]:
ypp = yp[0].reshape(-1,).tolist()[0] #PREDICTED LABELS in list form

In [24]:
from sklearn.metrics import classification_report

In [25]:
matrix = classification_report(yl,ypp)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
print(matrix)

              precision    recall  f1-score   support

           0       0.89      0.92      0.90       542
           1       0.77      0.79      0.78       592
           2       1.00      0.41      0.58        49
           3       0.84      0.77      0.80       239
           4       1.00      0.12      0.21        25
           5       0.69      0.65      0.67       228
           6       0.87      0.85      0.86       247
           7       0.82      0.77      0.79       465
           8       1.00      0.17      0.29        41
           9       0.93      0.78      0.85       104
          10       0.88      0.94      0.91       422
          11       0.86      0.69      0.77       248
          12       1.00      0.82      0.90        40
          13       0.74      0.76      0.75       543
          14       0.85      0.94      0.89       783
          15       0.96      0.94      0.95       529
          16       0.86      0.77      0.81       237
          17       0.88    

In [18]:
x_df = pd.DataFrame(x_np)
y_df = pd.DataFrame(y_np,columns=["Target"])
data = pd.concat([x_df, y_df], axis = 1)

In [19]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,Target
0,-0.024469,-0.067911,0.045744,0.066710,0.010599,0.045103,-0.009321,-0.060522,0.024939,-0.067728,-0.052724,0.028630,0.053921,-0.066996,-0.062883,-0.055374,0.040804,-0.051232,0.067534,0.019766,-0.010817,-0.011359,0.059858,-0.062840,0.038154,-0.043581,-0.067826,0.042609,-0.011737,0.013820,-0.041935,0.026769,0.063341,0.010446,-0.016767,-0.058427,0.013164,-0.003830,-0.002702,0.017349,-0.032754,0.036881,0.066355,0.019019,-0.038192,-0.061379,-0.057498,0.053939,0.005757,-0.060412,-0.030061,0.066802,-0.061295,0.015630,-0.057818,-0.048694,0.044784,-0.004273,0.052950,0.026517,-0.030392,0.021338,-0.036245,-0.029566,-0.060976,0.054291,0.029136,0.043816,0.030702,-0.032826,0.063022,0.067424,-0.003156,0.018902,-0.031083,-0.067916,-0.032648,0.024119,-0.032588,-0.065362,0.038731,0.053748,0.063137,0.003882,0.043497,-0.037704,-0.029949,0.031800,0.023282,-0.004645,0.033610,0.041356,-0.015773,-0.012157,0.036046,-0.024605,-0.020791,0.063884,0.053533,-0.064811,-0.055851,-0.061530,-0.059663,-0.034984,-0.047503,-0.059987,0.059046,-0.059241,-0.015462,0.037327,0.021940,0.056745,-0.032109,-0.033410,-0.000829,0.013181,-0.024244,-0.007296,0.065920,0.032092,0.033825,-0.002477,0.063719,0.053454,0.061416,-0.042120,-0.047913,0.030686,0.030499,0.055279,-0.066866,0.006412,0.063172,0.008990,0.010918,0.064034,-0.016224,0.028548,0.023565,0.017685,-0.060532,0.067900,-0.064467,0.061729,-0.029040,0.019282,-0.011600,0.025854,0.005815,-0.048947,-0.043324,-0.025888,-0.062599,0.017819,-0.037428,-0.052878,-0.010824,0.024019,0.013744,-0.048155,0.013036,-0.054724,-0.032846,0.039207,0.066847,0.017376,0.061404,-0.057195,-0.066646,-0.067822,0.059654,0.063884,0.065314,0.050279,0.020636,0.023530,0.025985,-0.061463,0.058304,-0.061956,0.063154,0.056900,0.039238,-0.054888,0.057398,0.027538,0.028767,-0.067664,0.067166,0.028901,0.015611,-0.062772,0.054893,-0.063281,0.005712,-0.057297,0.038086,0.019835,0.037907,0.058002,-0.048448,-0.028619,0.015281,0.055892,0.024748,0.014281,-0.011038,0.058299,0.029876,-0.043626,0.066305,0.064601,0.062420,0.067039,-0.021388,0.003346,-0.021024,0.059681,0.008064,-0.036416,0.045791,0.054247,-0.

In [2]:
!pip3 install jinja2
!pip3 install markupsafe==2.0.1
!pip3 install pycaret


  Using cached MarkupSafe-2.0.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (31 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


  Using cached MarkupSafe-2.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
#import jinja2
#pip install markupsafe==2.0.1
#!pip3 install --use-deprecated=legacy-resolver pycaret[full]
from pycaret.classification import *
import pycaret

     |████████████████████████████████| 53 kB 1.4 MB/s 
     |████████████████████████████████| 366 kB 9.7 MB/s 
     |████████████████████████████████| 305 kB 67.4 MB/s 
     |████████████████████████████████| 92 kB 116 kB/s 
     |████████████████████████████████| 132 kB 53.2 MB/s 
     |████████████████████████████████| 40 kB 5.3 MB/s 
     |████████████████████████████████| 76.6 MB 78.4 MB/s 
     |████████████████████████████████| 5.1 MB 27.0 MB/s 
     |████████████████████████████████| 308 kB 37.8 MB/s 
     |████████████████████████████████| 177 kB 43.9 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 100 kB 9.5 MB/s 
     |████████████████████████████████| 449 kB 47.3 MB/s 
     |████████████████████████████████| 11.9 MB 32.8 MB/s 
     |████████████████████████████████| 564 kB 50.7 MB/s 
     |████████████████████████████████| 53.2 MB 98 kB/s 
     |████████████████████████████████| 192.9 MB 52 kB/s 
     |█████████████████

In [20]:
clf = setup(data = data, target = 'Target',session_id=1229,normalize=True,transformation=True,ignore_low_variance=True,
           remove_multicollinearity=True, multicollinearity_threshold=0.95)

,Description,Value
0,session_id,1229
1,Target,Target
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(10000, 513)"
5,Missing Values,False
6,Numeric Features,512
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='Target',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strat...
                ('dummy', Dummify(target='Target')),
                ('fix_perfect', Remove_100(target='Target')),
                ('clean_names', Clean_Colum_Names()),
     

In [ ]:
compare_models(sort='Accuracy')

INFO:logs:Initializing compare_models()
INFO:logs:compare_models(include=None, fold=None, round=4, cross_validation=True, sort=Accuracy, n_select=1, budget_time=None, turbo=True, errors=ignore, fit_kwargs=None, groups=None, experiment_custom_tags=None, probability_threshold=None, verbose=True, display=None, exclude=None)
INFO:logs:Checking exceptions
INFO:logs:Preparing display monitor
INFO:logs:Preparing display monitor


IntProgress(value=0, description='Processing: ', max=84)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,03:43:01
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.7027,0.0000,0.5132,0.6901,0.6838,0.6823,0.6834,0.054
svm,SVM - Linear Kernel,0.6784,0.0000,0.5496,0.6829,0.6735,0.6574,0.6580,10.816
lr,Logistic Regression,0.6641,0.7665,0.5613,0.6671,0.6612,0.6432,0.6435,15.825
knn,K Neighbors Classifier,0.6611,0.7342,0.5331,0.6679,0.6574,0.6401,0.6407,3.268
rf,Random Forest Classifier,0.6561,0.7604,0.4577,0.6560,0.6335,0.6315,0.6338,7.013
nb,Naive Bayes,0.5822,0.7505,0.5530,0.6518,0.6012,0.5595,0.5612,0.094
dt,Decision Tree Classifier,0.4409,0.5650,0.3255,0.4449,0.4403,0.4071,0.4073,2.828
ada,Ada Boost Classifier,0.2662,0.4553,0.1303,0.1737,0.1780,0.1938,0.2377,10.999
qda,Quadratic Discriminant Analysis,0.2122,0.4552,0.0986,0.4217,0.1570,0.1215,0.2102,0.610


INFO:logs:Initializing Logistic Regression
INFO:logs:Total runtime is 8.50359598795573e-05 minutes
INFO:logs:SubProcess create_model() called ==================================
INFO:logs:Initializing create_model()
INFO:logs:create_model(estimator=lr, fold=StratifiedKFold(n_splits=10, random_state=None, shuffle=False), round=4, cross_validation=True, predict=True, fit_kwargs={}, groups=None, refit=False, verbose=False, system=False, metrics=None, experiment_custom_tags=None, add_to_model_list=True, probability_threshold=None, display=<pycaret.internal.Display.Display object at 0x7efadc085d50>, return_train_score=False, kwargs={})
INFO:logs:Checking exceptions
INFO:logs:Importing libraries
INFO:logs:Copying training dataset
INFO:logs:Defining folds
INFO:logs:Declaring metric variables
INFO:logs:Importing untrained model
INFO:logs:Logistic Regression Imported succesfully
INFO:logs:Starting cross validation
INFO:logs:Cross validating with StratifiedKFold(n_splits=10, random_state=None, sh